Going to try to identify all of the lakes in the NHD dataset. 

In [59]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
import seaborn as sns
import folium
%matplotlib inline

from IPython.display import HTML
def iframe(url):
    iframe = '<iframe src=' + url + ' width=1000 height=500></iframe>'
    return(HTML(iframe))

In [2]:
def basemap(projection=ccrs.Mercator.GOOGLE, extent=None):
    fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(projection=projection))
    states = NaturalEarthFeature(category='cultural', scale='50m', facecolor='none', name='admin_1_states_provinces_shp')
    if(extent is not None):
        ax.set_extent(extent)
    ax.add_feature(states)
    ax.gridlines()
    return fig, ax
    

In [3]:
datafile = "/Volumes/wrangell-st-elias/research/wa-shapefiles/Shape/NHDWaterbody.shp"
codesdbfile = "/Volumes/wrangell-st-elias/research/wa-shapefiles/Shape/NHDFCode.dbf"

In [4]:
nhd_waterbody = gpd.read_file(datafile)
print("%d waterbodies in %s" % (len(nhd_waterbody), datafile))

76817 waterbodies in /Volumes/wrangell-st-elias/research/wa-shapefiles/Shape/NHDWaterbody.shp


In [5]:
fcodes = gpd.read_file(codesdbfile)

In [6]:
nhd_waterbody.columns

Index(['AREASQKM', 'ELEVATION', 'FCODE', 'FDATE', 'FTYPE', 'GNIS_ID',
       'GNIS_NAME', 'PERMANENT_', 'REACHCODE', 'RESOLUTION', 'SHAPE_AREA',
       'SHAPE_LENG', 'VISIBILITY', 'geometry'],
      dtype='object')

In [7]:
fcodes.columns

Index(['CANALDITCH', 'CONSTRUCTI', 'DESCRIPTIO', 'FCODE', 'HYDROGRAPH',
       'INUNDATION', 'KCODE', 'OPERATIONA', 'PIPELINETY', 'POSITIONAL',
       'RELATIONSH', 'RESERVOIRT', 'STAGE', 'geometry'],
      dtype='object')

In [8]:
nhd_waterbody_coded = pd.merge(nhd_waterbody,
                               fcodes.drop('geometry', axis=1), 
                               left_on='FCODE', right_on='FCODE',
                               how='left')

In [9]:
print(nhd_waterbody_coded.AREASQKM.mean())
print(nhd_waterbody_coded.AREASQKM.std())

0.10712537279
5.43116852875


In [57]:
biglakes = nhd_waterbody_coded[(nhd_waterbody_coded.HYDROGRAPH == 'Perennial') & (nhd_waterbody_coded.AREASQKM > 0.1) & (nhd_waterbody_coded.AREASQKM < 20)]
#biglakes.plot(categorical=True, column='FTYPE', figsize=(14,6), edgecolor='None', cmap='Dark2', legend=True, alpha=1)
m = folium.Map([47.59, -120.3], zoom_start=8)
folium.GeoJson(biglakes.to_json(), name='biglakes').add_to(m)
m.save('test.html')
iframe('test.html')


KeyboardInterrupt: 

In [60]:
iframe('test.html')